# 01_compute_headmotion

Compute head movement and output the data for visualization

In [6]:
import numpy as np
from os.path import join as pjoin
import os
import mne_bids
import mne
from mne_bids import BIDSPath
import pandas as pd

In [7]:
# define variables
sub_list = ['{0:0>2d}'.format(sub) for sub in np.arange(1,12)]
run_list = ['{0:0>2d}'.format(run) for run in np.arange(1,9)]

# set path
bids_root = '/nfs/e5/studyforrest/forrest_movie_meg/gump_meg_bids'
results_pth = '/nfs/e5/studyforrest/forrest_movie_meg/tech_val_results'
if os.path.exists(results_pth) is False:
    os.mkdir(results_pth)

# set fiducials channel
coord = {'nas' : ['HLC0011','HLC0012','HLC0013'], 
         'lpa' : ['HLC0021','HLC0022','HLC0023'], 
         'rpa' : ['HLC0031','HLC0032','HLC0033']}
hpi_chns = list(np.concatenate(list(coord.values())))

In [8]:
def extract_hpi(raw_data):
    '''
    Extract hpi data from mne raw object.
    Returns
    -------
    hpi_data : Dict
        {channel_name: hpi_value}
    '''
    picks = mne.pick_channels_regexp(raw_data.ch_names, regexp='HLC00[123][123]...')
    hpi_data = raw_data.get_data(picks=picks)
    
    hpi = {}
    for i, pick in enumerate(picks):
        hpi[raw_data.ch_names[pick].split('-')[0]] = hpi_data[i]
            
    return hpi

In [9]:
def compute_headmotion(meg_data, coord):
    '''
    compute headmotion distance from mne raw object.
    Parameters
    ----------
    meg_data: mne object
    coord: Dict
        {fiducial: [channel name of x, channel name of y, channel name of z]}
    
    Returns
    -------
    head_movement : array
        [n_fiducial, n_sample]
    '''
    
    raw_data = meg_data.copy().crop(tmin=meg_data.annotations.onset[0], tmax=meg_data.annotations.onset[-1])
    hpi = extract_hpi(raw_data)
    
    # get relative positions to initial positions
    hpi_de = {}
    for chn, val in hpi.items():
        hpi_de[chn] = 1000*(val-val[0])
    
    # compute head motion
    head_movement = [np.sqrt(hpi_de[coord[fiducial][0]]**2 + 
                             hpi_de[coord[fiducial][1]]**2 + 
                             hpi_de[coord[fiducial][2]]**2) 
                     for fiducial in coord.keys()]
    
    return np.asarray(head_movement)

In [ ]:
# get headmotion distances - within run
head_motion = {}
for sub in sub_list:
    
    # get runlist
    head_motion[sub] = []
    if sub == '01':
        run_ls = run_list + ['09']
    else:
        run_ls = run_list
        
    for run in run_ls:
        # get hpi data
        sub_path = BIDSPath(subject=sub, run=int(run), task='movie', session='movie', root=bids_root)
        raw = mne_bids.read_raw_bids(sub_path)
        
        head_motion[sub].append(compute_headmotion(raw, coord))

In [ ]:
# convert head motion data to 1000 bins
# get max head movement
upper_bound = np.ceil(max([run.max() for sub, sub_data in head_motion.items() for run in sub_data]))

# binning
hm = {}
for sub in sub_list:
    if sub == '01':
        run_ls = run_list + ['09']
    else:
        run_ls = run_list
    
    hm_sub = []
    for run in run_ls:
        
        mv = head_motion[sub][int(run)-1]
        hm_bins = [pd.cut(mv[i], np.linspace(0, upper_bound, 1000)).value_counts().values 
                   for i in np.arange(mv.shape[0])]
        
        hm_sub.append((np.asarray(hm_bins),np.max(mv, axis=1)))
        
    hm[sub] = hm_sub
    print( sub + ' done')

In [7]:
# save headmotion data - within run
for sub in list(hm.keys())[1:]:
    hm[sub].append(np.nan)
df = pd.DataFrame(hm, columns=sub_list, index=run_list+['09'])
df.to_pickle(pjoin(results_pth, 'head_motion.pickle'))

np.save(pjoin(results_pth, 'head_motion_bins.npy'), np.linspace(0, upper_bound, 1000))

In [ ]:
# get headmotion - between runs / during break
break_deviation = pd.DataFrame(columns=coord.keys(), index=sub_list)
for j in np.arange(break_deviation.shape[1]):
    for i in np.arange(break_deviation.shape[0]):
        break_deviation.iloc[i,j] = []

run_deviation = pd.DataFrame(columns=coord.keys(), index=sub_list)
for j in np.arange(run_deviation.shape[1]):
    for i in np.arange(run_deviation.shape[0]):
        run_deviation.iloc[i,j] = []

head_motion = {}
for sub in sub_list:
    
    # get runlist
    head_motion[sub] = []
    if sub == '01':
        run_ls = run_list + ['09']
    else:
        run_ls = run_list
    
    initial_pos = pd.DataFrame(columns=hpi_chns, index=run_ls)
    last_pos = pd.DataFrame(columns=hpi_chns, index=run_ls)
    for run in run_ls:
        # get hpi data
        sub_path = BIDSPath(subject=sub, run=int(run), task='movie', session='movie', root=bids_root)
        raw = mne_bids.read_raw_bids(sub_path)
        raw = raw.copy().crop(tmin=raw.annotations.onset[0], tmax=raw.annotations.onset[-1])
        hpi = extract_hpi(raw)
        for chn in hpi_chns:
            initial_pos[chn][run] = hpi[chn][0]
            last_pos[chn][run] = hpi[chn][-1]
    
    # compute head movement during breaks
    for i in range(len(initial_pos)-1):
        break_dist = 1000*(initial_pos.iloc[i+1,:] - last_pos.iloc[i,:])
        for fiducial in coord.keys():
            break_deviation[fiducial][sub].append(np.sqrt(break_dist[coord[fiducial][0]]**2 + 
                                    break_dist[coord[fiducial][1]]**2 + 
                                    break_dist[coord[fiducial][2]]**2))

    # compute head movement from initial position
    fst_run_pos = initial_pos.iloc[0,:]
    for i, row in enumerate(initial_pos.iterrows()):
        run_dist = 1000*(row[1] - fst_run_pos)
        for fiducial in coord.keys():
            run_deviation[fiducial][sub].append(np.sqrt(run_dist[coord[fiducial][0]]**2 + 
                                    run_dist[coord[fiducial][1]]**2 + 
                                    run_dist[coord[fiducial][2]]**2))

# save
break_deviation.to_pickle(pjoin(results_pth, 'break_deviation.pickle'))
run_deviation.to_pickle(pjoin(results_pth, 'run_deviation.pickle'))